In [ ]:
# Check if running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    # Clone repository and install dependencies
    !git clone https://github.com/khaytao/LocCCA/
    %cd LocCCA
    !pip install -r requirements.txt

    

In [6]:
import json

# Load config file
with open('config/colab_configs.json', 'r') as f:
    configs = json.load(f)

    print(configs["tasks"])


['data-generation', 'train', 'eval']


In [ ]:
# Run data generation if enabled in config
if "data-generation" in configs["tasks"]:
    data_gen_args = configs["data-generation"]
    cmd = f"python dataset_generation.py"
    
    # Add all arguments from config
    cmd += f" --room_dim {data_gen_args['room_dim'][0]} {data_gen_args['room_dim'][1]} {data_gen_args['room_dim'][2]}"
    cmd += f" --t60 {data_gen_args['t60']}"
    cmd += f" --fs {data_gen_args['fs']}" 
    cmd += f" --mic_height {data_gen_args['mic_height']}"
    cmd += f" --array_distance {data_gen_args['array_distance']}"
    cmd += f" --height {data_gen_args['height']}"
    cmd += f" --num_speakers {data_gen_args['num_speakers']}"
    cmd += f" --distribution {data_gen_args['distribution']}"
    cmd += f" --centroids {data_gen_args['centroids'][0]} {data_gen_args['centroids'][1]}"
    cmd += f" --std {data_gen_args['std']}"
    cmd += f" --data_location {data_gen_args['data_location']}"
    cmd += f" --dataset_name {data_gen_args['dataset_name']}"
    cmd += f" --room_resolution {data_gen_args['room_resolution']}"
    
    !{cmd}


In [7]:
# Run evaluation if enabled in config
if "eval" in configs["tasks"]:
    eval_args = configs["eval"]
    cmd = f"python eval_model.py"
    
    # Add all arguments from config
    cmd += f" --model {eval_args['model']}"
    cmd += f" --data_dir {eval_args['data_dir']}" 
    cmd += f" --threshold {eval_args['threshold']}"
    cmd += f" --plot {eval_args['plot']}"
    
    !{cmd}


Figure(640x480)
Number of failures: 5
MSE: 0.02382950857281685


INFO:speechbrain.utils.quirks:Applied quirks (see `speechbrain.utils.quirks`): [allow_tf32, disable_jit_profiling]
INFO:speechbrain.utils.quirks:Excluded quirks specified by the `SB_DISABLE_QUIRKS` environment (comma-separated list): []

100%|██████████| 10/10 [00:04<00:00,  2.33it/s]
